In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import collections

### Creating model 

In [2]:
class Yolo(object) :
    def __init__(self,weights_path, cfg_path, coco_path) :
        self.net= cv2.dnn.readNet(weights_path, cfg_path)
        self.regions=[
                    [27,129,230,289] ,
                    [287,156,711,431],
                    [10,8,755,566],   
                    ]
        self.out = None 
        self.cap = None
        self.classes=self.setClasses(coco_path)
        
    def checkRegion(self, x_center, y_center) :
        """
        returns the region that the bbox belongs to (0 if none)
        """
        for i,region in enumerate(self.regions) :
            if region[0] <= x_center <=region[2] and region[1]<=y_center<=region[3] :
                    return i+1
        return 0
    
    def drawRegions(self,image):
        colors=[(0, 0, 255),(0, 255, 0),(255, 0, 0)]
        for i,region in enumerate(self.regions):
            cv2.rectangle(image, (region[0],region[1]), (region[2],region[3]), colors[i], 2)
    
    def drawCounts(self,regions_count,image) :
        #Region-3 count(smallest)
        cv2.putText(image, 'Count = '+str(regions_count[1]), (100,280), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 2)
        #Region-2 count(mid)
        cv2.putText(image, 'Count = '+str(regions_count[2]), (450,410), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 255, 0), 2)
        #Region-1 count(largest)
        cv2.putText(image, 'Count = '+str(regions_count[1]+regions_count[2]+regions_count[3]), (400,40), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 0), 2)
        
    def setClasses(self,path) :
        """
        return  classes names that yolo can detect
        """
        with open(path, 'r') as f:
            classes = [line.strip() for line in f.readlines()]
        return classes
    
    def detect(self,imagesPath,outputPath) :
        self.cap=cv2.VideoCapture(imagesPath)
        self.out=cv2.VideoWriter(outputPath + 'output.mp4',cv2.VideoWriter_fourcc('m','p','4','v'), 6, (768,576))
        
        while 1 :
            ret, image = self.cap.read()
            if ret:
                self.net.setInput(cv2.dnn.blobFromImage(image, 0.00392, (416,416), (0,0,0), True, crop=False))
                layer_names = self.net.getLayerNames()
                output_layers = [layer_names[i[0] - 1] for i in self.net.getUnconnectedOutLayers()]
                outs = self.net.forward(output_layers)
                
                class_ids = []
                confidences = []
                boxes = []
                boxes_centers=[]
                Width = image.shape[1]
                Height = image.shape[0]

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.5:
                            center_x = int(detection[0] * Width)
                            center_y = int(detection[1] * Height)
                            
                            w = int(detection[2] * Width)
                            h = int(detection[3] * Height)
                            
                            x = center_x - w / 2
                            y = center_y - h / 2
                            
                            class_ids.append(class_id)
                            confidences.append(float(confidence))
                            boxes.append([x, y, w, h])
                            boxes_centers.append([center_x,center_y])

                indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.1)
                regions_count=collections.defaultdict(int)
                for i in indices:
                    i = i[0]
                    box = boxes[i]
                    #check if person detection
                    if class_ids[i]==0:
                            region= self.checkRegion(*boxes_centers[i])
                            if region :
                                label = str(self.classes[class_id]) 
                                cv2.rectangle(image, (round(box[0]),round(box[1])), (round(box[0]+box[2]),round(box[1]+box[3])), (0, 0, 0), 2)
                                cv2.putText(image, label, (round(box[0])-10,round(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
                                regions_count[region]+=1
                self.drawRegions(image)
                self.drawCounts(regions_count,image)
                cv2.imshow('frame',image)
                self.out.write(image)
                if cv2.waitKey(1) == ord('q'):
                        break
                        
            else : 
                break
                
        self.cap.release() 
        self.out.release()
        cv2.destroyAllWindows()        

### Creating output directories and input data paths 

In [3]:
input_paths=[]
for i in range(1,5) :
    path1= "./S1_L1/Crowd_PETS09/S1/L1/Time_13-57/View_00{}/frame_%04d.jpg".format(i)
    path2='./S1_L1/Crowd_PETS09/S1/L1/Time_13-59/View_00{}/frame_%04d.jpg'.format(i)
    input_paths.extend([path1,path2])

In [4]:
output_paths=[]
for i in range(1,5) :
    path1="S1_L1_output/Crowd_PETS09/S1/L1/Time_13-57_output/View_00{}_output/".format(i)
    path2="S1_L1_output/Crowd_PETS09/S1/L1/Time_13-59_output/View_00{}_output/".format(i)
    output_paths.extend([path1,path2])

In [5]:
for path in output_paths :
    try :
        os.makedirs(path)
    except :
        print(path+' is already created')

### Intializing and running the model

In [6]:
weights_path='./Yolo_files/yolov3.weights'
cfg_path='./Yolo_files/yolov3.cfg'
names_path='./Yolo_files/coco.names'


In [7]:
yolo=Yolo(weights_path, cfg_path, names_path)

In [8]:
# Read each input frame from input_path and saves it's output in output_path 
for input_path,output_path in zip (input_paths,output_paths) :
    yolo.detect(input_path,output_path)